# Chess Logic W/JS
> Here's an example of my chess logic with Javascript

- layout: default
- badges: false
- toc: false
- categories: [week_8]

In [1]:
chessBoard = {};

lettersOnBoard = "abcdefgh";



function printBoard(){
    for (j = 0; j <= 7; j++){
        letter = lettersOnBoard[j];
        console.log("")
        for (i = 1; i <= 8; i++){
            var newKey = letter + i;
            process.stdout.write(chessBoard[newKey][0] + " ");
        }
    }
}




'abcdefgh'

In [2]:

lettersOnBoard = "abcdefgh";
chessBoard = {};

for (j = 0; j <= 7; j++){
    letter = lettersOnBoard[j];
    for (i = 1; i <= 8; i++){
        var newKey = letter + i;
        chessBoard[newKey] = ["OO", undefined]
    }
}


// piece class, to be extended by other classes
class piece{
    constructor(_position, _color){
        this.position = _position;
        this.color = _color
    }
    
    move(move, currentM){
        let currentBoard = chessBoard[currentM];
        if(this.getFreeMoves().includes(move)) {
            this.position = move;
            chessBoard[move] = currentBoard;
            chessBoard[currentM] = ["OO", undefined];
        }
    }
}


[ 'OO', undefined ]

In [3]:
class rook extends piece{
    
    constructor(_position, _color){
        // super is the position constructor, uh basically does some super cool inheritence stuff or something. 
        super(_position, _color);
        // automatically sets the spot on the board which is passed in to this rook
        this.id = "R"
        }   
    //method to return all of the available moves that the piece can make. 
    getTotalMoves(){
        let currentPosition = this.position.split("");
        let moves = [];

        for (var i = 1; i <= 8; i++){
            var newMove = currentPosition[0] + i;
            moves[i - 1] = newMove;
        }
        for (var i = 1; i <= 8; i++){
            var newMove = lettersOnBoard[i - 1] + currentPosition[1];
            moves.push(newMove);
        }
        
        let totalMoves = [];

        moves.forEach((c) => {
            if (c != this.position){
                totalMoves.push(c);
            }
        });

        return totalMoves;
    }
    
    //method to return all of the obstructed moves based on the total moves
    getObstructedMoves(){
        let totalMoves = this.getTotalMoves();
        let obstructedMoves = [];
        let blockedMoves = [];
        let index = -1

        totalMoves.forEach((c) => {
            if (!obstructedMoves.includes(c)){
                if (chessBoard[c][0] != "OO"){
                    obstructedMoves.push(c);
                    index++
                    totalMoves.forEach((c) => {
                        try{
                            if (obstructedMoves[index][1] > this.position[1] && c[1] > obstructedMoves[index][1]){
                                blockedMoves.push(c)
                            }
                            else if (obstructedMoves[index][1] < this.position[1] && c[1] < obstructedMoves[index][1]){
                                blockedMoves.push(c)
                            }
                        } catch{}
                        try{
                            if (lettersOnBoard.indexOf(obstructedMoves[index][0]) > lettersOnBoard.indexOf(this.position[0]) && lettersOnBoard.indexOf(c[0]) > lettersOnBoard.indexOf(obstructedMoves[index][0])){
                                blockedMoves.push(c)
                            }
                            if (lettersOnBoard.indexOf(obstructedMoves[index][0]) < lettersOnBoard.indexOf(this.position[0]) && lettersOnBoard.indexOf(c[0]) < lettersOnBoard.indexOf(obstructedMoves[index][0])){
                                blockedMoves.push(c)
                            }
                        } catch{}
                    })
                } 
            }
        })

        blockedMoves.forEach((c) => {obstructedMoves.push(c);})
        return obstructedMoves;
    }

    //method to return all of the moves which are not obstructed
    getFreeMoves(){
        let totalMoves = this.getTotalMoves();
        let obstructedMoves = this.getObstructedMoves();

        totalMoves = totalMoves.filter( (c) => !obstructedMoves.includes(c) );

        return totalMoves;
    }

    //method to return the pieces which can be captured. 
    getAvailableCaptures(){
        // defines new variables as other methods in this class which may be useful.
        let totalMoves = this.getTotalMoves();
        let obstructedMoves = this.getObstructedMoves();
        // defines arrays
        let sameRow = [];
        let sameColumn = [];
        let columnNums = [];
        let columnDifs = [];
        let negDifsColumn = [];
        let posDifsColumn = [];
        let rowLets = [];
        let rowNums = [];
        let rowDifs = [];
        let posDifsRow = [];
        let negDifsRow = [];
        let captures = [];
        let finalCaptures = [];
        // finds all of the moves which are in the same row or in the same column as the rook.
        obstructedMoves.forEach((c) => {
            if (this.position.split("")[0] == c.split("")[0]){
                sameColumn.push(c);
            }
            else if (this.position.split("")[1] == c.split("")[1]){
                sameRow.push(c);
            }
        })
        //adds to a new array all of the numbers in the obstructed columns. Also converts it to an Integer
        sameColumn.forEach((c) => {
            columnNums.push(parseInt(c.split("")[1]));
        })        
        //elipses is a spread function, basically inputs each value in the array as it's own parameter. 
        //this foreach finds the difference between the pieces in the same column and the rooks current position
        columnNums.forEach((c) => {
            columnDifs.push(c - parseInt(this.position.split("")[1]))
        });
        //this foreach defines two new
        columnDifs.forEach((c) => {
            if (c < 0) negDifsColumn.push(Math.abs(c)); else posDifsColumn.push(c);
        })
        // finds the minimum positive number and the minimum negative number and converts them to an integer
        var posMinColumn = parseInt(Math.min(...posDifsColumn));
        var negMinColumn = parseInt(Math.min(...negDifsColumn));
        // works backwards to find the position on the board given the smallest differences 
        sameColumn.forEach((c) => {
            if (parseInt(c.split("")[1]) == parseInt(this.position.split("")[1]) + posMinColumn || parseInt(c.split("")[1]) == parseInt(this.position.split("")[1]) - negMinColumn){
                captures.push(c)
            }
        })
        // basically does all of the same stuff but for the rows using the index of the lettersOnBoard array
        sameRow.forEach((c) => {
            rowLets.push(c.split("")[0]);
        })
        rowLets.forEach((c) => {
            rowNums.push(lettersOnBoard.indexOf(c) + 1)
        })
        rowNums.forEach((c) => {
            rowDifs.push(parseInt(c) - (lettersOnBoard.indexOf(this.position.split("")[0]) + 1))
        })
        rowDifs.forEach((c) => {
            if (c < 0) negDifsRow.push(Math.abs(c)); else posDifsRow.push(c);
        })
        var posMinRow = parseInt(Math.min(...posDifsRow));
        var negMinRow = parseInt(Math.min(...negDifsRow))
        sameRow.forEach((c) => {
            if ((lettersOnBoard.indexOf(c.split("")[0]) + 1) == (lettersOnBoard.indexOf(this.position.split("")[0]) + posMinRow + 1) || (lettersOnBoard.indexOf(c.split("")[0]) + 1) == (lettersOnBoard.indexOf(this.position.split("")[0]) - negMinRow + 1)){
                captures.push(c)
            }
        })
        //checks if captures are the same color or not
        captures.forEach((c) => {
            if (chessBoard[c][0].split("")[0] != this.color){
                finalCaptures.push(c);
            }
        })
        return finalCaptures
    }
    getAvailableMoves(){
        let freeMoves = this.getFreeMoves()
        let captures = this.getAvailableCaptures()
        captures.forEach((c) => {
            freeMoves.push(c);
        })
        return freeMoves;
    }
    
}

let rook1 = new rook("d5", "b");
chessBoard[rook1.position] = [rook1.color + rook1.id, rook1]

let rook2 = new rook("e5", "w")
chessBoard[rook2.position] = [rook2.color + rook2.id, rook2]

let rook3 = new rook("f5", "w")
chessBoard[rook3.position] = [rook3.color + rook3.id, rook3]

printBoard()

console.log("")
console.log(rook1.getAvailableMoves())

//console.log(rook1.getFreeMoves())




OO OO OO OO OO OO OO OO 
OO OO OO OO OO OO OO OO 
OO OO OO OO OO OO OO OO 
OO OO OO OO bR OO OO OO 
OO OO OO OO wR OO OO OO 
OO OO OO OO wR OO OO OO 
OO OO OO OO OO OO OO OO 
OO OO OO OO OO OO OO OO 
[ 'd1', 'd2', 'd3', 'd4', 'd6', 'd7', 'd8', 'a5', 'b5', 'c5', 'e5' ]


In [4]:
class pawn extends piece{
    constructor(_position, _color){
        // super is the position constructor, uh basically does some super cool inheritence stuff or something. 
        super(_position, _color);
        // automatically sets the spot on the board which is passed in to this pawn using the parent method
        if (_color == "w"){this.direction = 1}
        else if (_color == "b") {this.direction = -1}
        this.hasMoved = 0;
        this.id = "P";
    }
    move(move, currentM){
        super.move(move, currentM)
        this.hasMoved = 1
    }
    getTotalMoves(){
        let moves = [];
        let currentPosition = this.position.split("");

        if(this.hasMoved == 0){
            moves.push(currentPosition[0] + (parseInt(currentPosition[1]) + (1 * this.direction)))
            moves.push(currentPosition[0] + (parseInt(currentPosition[1]) + (2 * this.direction)))
        }
        else{
            moves.push(currentPosition[0] + (parseInt(currentPosition[1]) + (1 * this.direction)))
        }
        return moves;
    }
    getFreeMoves(){
        let moves = this.getTotalMoves();
        let freeMoves = [];

        if (chessBoard[moves[0]][0] == "OO"){
            freeMoves.push(moves[0]);
            try{
                if (chessBoard[moves[1]][0] == "OO"){freeMoves.push(moves[1]);}
            }catch{}
        }
        return freeMoves;
    } 
    getAvailableCaptures(){
        let captures = [];
        let currentPosition = this.position.split(""); 
        let possibleLets = [
            lettersOnBoard[lettersOnBoard.indexOf(currentPosition[0]) - 1],
            lettersOnBoard[lettersOnBoard.indexOf(currentPosition[0]) + 1]
        ];
        possibleLets = possibleLets.filter(c => c != undefined);
        possibleLets.forEach((c) => {
            let a = parseInt(currentPosition[1]) + (1 * this.direction)
            let check = c + a
            if (chessBoard[check][0][0] != this.color && chessBoard[check][0] != "OO"){
                    captures.push(check);
            }
        })
        console.log(captures);
    }
}

let pawn1 = new pawn("c5", "b")



// for (var i = 0; i < 8; i++){
//     let i = new pawn ("c" + toString(i), b)
// }

//console.log(chessBoard[])

In [5]:
class king extends piece{
    
    constructor(_position, _color){
        // super is the position constructor, uh basically does some super cool inheritence stuff or something. 
        super(_position, _color);
        // automatically sets the spot on the board which is passed in to this rook
    }
    //method to return all of the available moves that the piece can make. 
    getTotalMoves(){
        let currentPosition = this.position.split("");
        let moves = [];

        let aboveNum = parseInt(currentPosition[1]) + 1
        let currentLetNum = lettersOnBoard.indexOf(currentPosition[0])
        
        for (var i = -1; i <= 1; i++){
            moves.push(lettersOnBoard[currentLetNum + i] + aboveNum);
        }

        let belowNum = parseInt(currentPosition[1]) - 1

        for (var i = -1; i <= 1; i++){
            moves.push(lettersOnBoard[currentLetNum + i] + belowNum);
        }
        
        let sameNum = parseInt(currentPosition[1])

        for (var i = -1; i <= 1;i++){
            if (i != 0){
                moves.push(lettersOnBoard[currentLetNum + i] + sameNum);
            }
        }

        return moves;
    }

    //method to return all of the obstructed moves based on the total moves
    getObstructedMoves(){
        let totalMoves = this.getTotalMoves();
        let obstructedMoves = [];
        for (var i = 0; i < totalMoves.length; i++){
            if (chessBoard[totalMoves[i]][0] != "OO"){
                obstructedMoves.push(totalMoves[i]);
            }
        }
        return obstructedMoves;
    }

    //method to return all of the moves which are not obstructed
    getFreeMoves(){
        let totalMoves = this.getTotalMoves();
        let obstructedMoves = this.getObstructedMoves();

        totalMoves.forEach((c) => {
            //console.log(c);
            for (var i = 0; i < obstructedMoves.length; i ++){
                if (c == obstructedMoves[i]){
                    totalMoves.splice(totalMoves.indexOf(c), 1);
                }
            }
        })

        return totalMoves;
    }
    

    //method to return the pieces which can be captured. 
    getAvailableCaptures(){
        // defines new variables as other methods in this class which may be useful.
        let totalMoves = this.getTotalMoves();
        let obstructedMoves = this.getObstructedMoves();
        // defines arrays
        let sameRow = [];
        let sameColumn = [];
        let columnNums = [];
        let columnDifs = [];
        let negDifsColumn = [];
        let posDifsColumn = [];
        let rowLets = [];
        let rowNums = [];
        let rowDifs = [];
        let posDifsRow = [];
        let negDifsRow = [];
        let captures = [];
        let finalCaptures = [];
        // finds all of the moves which are in the same row or in the same column as the rook.
        obstructedMoves.forEach((c) => {
            if (this.position.split("")[0] == c.split("")[0]){
                sameColumn.push(c);
            }
            else if (this.position.split("")[1] == c.split("")[1]){
                sameRow.push(c);
            }
        })
        //adds to a new array all of the numbers in the obstructed columns. Also converts it to an Integer
        sameColumn.forEach((c) => {
            columnNums.push(parseInt(c.split("")[1]));
        })        
        //elipses is a spread function, basically inputs each value in the array as it's own parameter. 
        //this foreach finds the difference between the pieces in the same column and the rooks current position
        columnNums.forEach((c) => {columnDifs.push(c - parseInt(this.position.split("")[1]))});
        //this foreach defines two new
        columnDifs.forEach((c) => {if (c < 0) negDifsColumn.push(Math.abs(c)); else posDifsColumn.push(c);})
        // finds the minimum positive number and the minimum negative number and converts them to an integer
        var posMinColumn = parseInt(Math.min(...posDifsColumn));
        var negMinColumn = parseInt(Math.min(...negDifsColumn));
        // works backwards to find the position on the board given the smallest differences 
        sameColumn.forEach((c) => {if (parseInt(c.split("")[1]) == parseInt(this.position.split("")[1]) + posMinColumn || parseInt(c.split("")[1]) == parseInt(this.position.split("")[1]) - negMinColumn){captures.push(c)}})
        // basically does all of the same stuff but for the rows using the index of the lettersOnBoard array
        sameRow.forEach((c) => {rowLets.push(c.split("")[0]);})
        rowLets.forEach((c) => {rowNums.push(lettersOnBoard.indexOf(c) + 1)})
        rowNums.forEach((c) => {rowDifs.push(parseInt(c) - (lettersOnBoard.indexOf(this.position.split("")[0]) + 1))})
        rowDifs.forEach((c) => {if (c < 0) negDifsRow.push(Math.abs(c)); else posDifsRow.push(c);})
        var posMinRow = parseInt(Math.min(...posDifsRow));
        var negMinRow = parseInt(Math.min(...negDifsRow))
        sameRow.forEach((c) => {if ((lettersOnBoard.indexOf(c.split("")[0]) + 1) == (lettersOnBoard.indexOf(this.position.split("")[0]) + posMinRow + 1) || (lettersOnBoard.indexOf(c.split("")[0]) + 1) == (lettersOnBoard.indexOf(this.position.split("")[0]) - negMinRow + 1)){captures.push(c)}})
        //checks if captures are the same color or not
        captures.forEach((c) => {
            if (chessBoard[c].split("")[0] != this.color){
                finalCaptures.push(c);
            }
        })
        return finalCaptures
    }
    
}

let king1 = new king("d5", "b");

console.log(king1.getObstructedMoves())


[ 'e5' ]


In [6]:
let rookb1 = new rook("a8", "b");
setBoard(rookb1)

let rookb2 = new rook("h8", "b");
setBoard(rookb2)

let rookw1 = new rook("a1", "w");
setBoard(rookw1)

let rookw2 = new rook("h1", "w");
setBoard(rookw2)

let pawnw1 = new pawn("a2", "w")
setBoard(pawnw1)

let pawnw2 = new pawn("b2", "w")
setBoard(pawnw2)

let pawnw3 = new pawn("c2", "w")
setBoard(pawnw3)

let pawnw4 = new pawn("d2", "w")
setBoard(pawnw4)

let pawnw5 = new pawn("e2", "w")
setBoard(pawnw5)

let pawnw6 = new pawn("f2", "w")
setBoard(pawnw6)

let pawnw7 = new pawn("g2", "w")
setBoard(pawnw7)

let pawnw8 = new pawn("h2", "w")
setBoard(pawnw8)

// printBoard()
// movePiece("g2", "g4");
// movePiece("a1", "g1");
// console.log("");
// movePiece("g4", "g5");
// movePiece("g5", "g6");
// movePiece("g6", "g7");
// printMoves("g7")
// movePiece("g7", "h8");
// printBoard();

console.log(chessBoard["a1"]);
console.log(chessBoard["a1"][1].getFreeMoves());

function setBoard(obj){
    chessBoard[obj.position] = [obj.color + obj.id, obj]
}

function movePiece(currentM, newM){
    chessBoard[currentM][1].move(newM, currentM)
}

function printMoves(current){
    console.log(chessBoard[current][1].getAvailableMoves())
}

[ 'wR', rook { position: 'a1', color: 'w', id: 'R' } ]
[ 'b1', 'c1', 'd1', 'e1', 'f1', 'g1' ]
